# DRW - Crypto Market Prediction : Dimensional Reduction Notebook

### Dependencies

In [17]:
import re
import os
import json
import numpy as np
import pandas as pd
import xgboost as xgb

### Preparing the Data

In [18]:
train_df = pd.read_parquet(os.path.join("Data","train.parquet"))
train_df.head()

,bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,...,X882,X883,X884,X885,X886,X887,X888,X889,X890,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-03-01 00:00:00,15.283,8.425,176.405,44.984,221.389,0.121263,-0.417690,0.005399,0.125948,0.058359,...,1.925423,1.847943,0.005676,0.190791,0.369691,0.377630,0.210153,0.159183,0.530636,0.562539
2023-03-01 00:01:00,38.590,2.336,525.846,321.950,847.796,0.302841,-0.049576,0.356667,0.481087,0.237954,...,1.928569,1.849468,0.005227,0.184660,0.363642,0.374515,0.209573,0.158963,0.530269,0.533686
2023-03-01 00:02:00,0.442,60.250,159.227,136.369,295.596,0.167462,-0.291212,0.083138,0.206881,0.101727,...,1.928047,1.849282,0.004796,0.178719,0.357689,0.371424,0.208993,0.158744,0.529901,0.546505
2023-03-01 00:03:00,4.865,21.016,335.742,124.963,460.705,0.072944,-0.436590,-0.102483,0.017551,0.007149,...,1.928621,1.849608,0.004398,0.172967,0.351832,0.368358,0.208416,0.158524,0.529534,0.357703
2023-03-01 00:04:00,27.158,3.451,98.411,44.407,142.818,0.173820,-0.213489,0.096067,0.215709,0.107133,...,1.927084,1.848950,0.004008,0.167391,0.346066,0.365314,0.207839,0.158304,0.529167,0.362452


In [3]:
train_df.replace([np.inf,-np.inf],np.nan,inplace=True)
train_df.head()

,bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,...,X882,X883,X884,X885,X886,X887,X888,X889,X890,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-03-01 00:00:00,15.283,8.425,176.405,44.984,221.389,0.121263,-0.417690,0.005399,0.125948,0.058359,...,1.925423,1.847943,0.005676,0.190791,0.369691,0.377630,0.210153,0.159183,0.530636,0.562539
2023-03-01 00:01:00,38.590,2.336,525.846,321.950,847.796,0.302841,-0.049576,0.356667,0.481087,0.237954,...,1.928569,1.849468,0.005227,0.184660,0.363642,0.374515,0.209573,0.158963,0.530269,0.533686
2023-03-01 00:02:00,0.442,60.250,159.227,136.369,295.596,0.167462,-0.291212,0.083138,0.206881,0.101727,...,1.928047,1.849282,0.004796,0.178719,0.357689,0.371424,0.208993,0.158744,0.529901,0.546505
2023-03-01 00:03:00,4.865,21.016,335.742,124.963,460.705,0.072944,-0.436590,-0.102483,0.017551,0.007149,...,1.928621,1.849608,0.004398,0.172967,0.351832,0.368358,0.208416,0.158524,0.529534,0.357703
2023-03-01 00:04:00,27.158,3.451,98.411,44.407,142.818,0.173820,-0.213489,0.096067,0.215709,0.107133,...,1.927084,1.848950,0.004008,0.167391,0.346066,0.365314,0.207839,0.158304,0.529167,0.362452


In [4]:
train_df.describe()

,bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,...,X882,X883,X884,X885,X886,X887,X888,X889,X890,label
count,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000,...,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000,525887.000000
mean,9.968003,10.174169,131.726678,132.673944,264.400622,-0.006026,-0.000243,-0.000353,-0.000425,-0.000263,...,1.959330,1.954978,0.498112,0.498063,0.497997,0.497872,0.497525,0.497372,0.995394,0.036126
std,15.645741,15.889582,307.267251,309.803040,588.618746,0.538340,0.613746,0.771744,0.857115,0.464695,...,0.844732,0.598279,0.712811,0.698678,0.684946,0.666936,0.615440,0.546439,0.850509,1.009914
min,0.001000,0.001000,0.000000,0.000000,0.000000,-2.787539,-5.861926,-6.125439,-6.093792,-2.855040,...,0.433416,0.686524,0.000004,0.000006,0.000009,0.000024,0.003903,0.015654,0.092913,-24.416615
25%,2.634000,2.678000,26.407000,27.021000,60.688500,-0.370635,-0.314135,-0.441536,-0.521693,-0.295502,...,1.367222,1.521202,0.000162,0.000193,0.000241,0.000997,0.033156,0.066908,0.244373,-0.381585
50%,6.415000,6.538000,57.015000,58.047000,120.799000,-0.015991,-0.001713,-0.003010,-0.003948,-0.002545,...,1.831805,1.882957,0.000634,0.001036,0.004593,0.039178,0.127865,0.161757,0.634448,0.016262
75%,13.085000,13.330000,127.639000,129.110000,256.733500,0.349104,0.310181,0.431960,0.507830,0.287728,...,2.369648,2.274296,1.473225,1.472025,1.442417,1.380184,1.120480,0.999344,1.701838,0.434135
max,1114.932000,1352.965000,17614.400000,17686.234000,28701.419000,2.972741,6.151366,6.488532,6.394355,3.184776,...,6.881902,4.745380,1.584315,1.584315,1.584315,1.584315,1.583862,1.557910,2.769604,20.740270


In [6]:
train_df.dtypes

bid_qty     float64
ask_qty     float64
buy_qty     float64
sell_qty    float64
volume      float64
             ...   
X887        float64
X888        float64
X889        float64
X890        float64
label       float64
Length: 896, dtype: object

### Feature Compression

In [7]:
def Preprocessing(df):
    df.drop(columns = ["bid_qty","ask_qty","sell_qty","volume","buy_qty"],inplace = True)
    return df

In [8]:
processed_train_df = Preprocessing(train_df)
processed_train_df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X882,X883,X884,X885,X886,X887,X888,X889,X890,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-03-01 00:00:00,0.121263,-0.417690,0.005399,0.125948,0.058359,0.027359,0.035780,0.068219,1.034825,-0.029575,...,1.925423,1.847943,0.005676,0.190791,0.369691,0.377630,0.210153,0.159183,0.530636,0.562539
2023-03-01 00:01:00,0.302841,-0.049576,0.356667,0.481087,0.237954,0.208359,0.217057,0.249624,0.948694,-0.183488,...,1.928569,1.849468,0.005227,0.184660,0.363642,0.374515,0.209573,0.158963,0.530269,0.533686
2023-03-01 00:02:00,0.167462,-0.291212,0.083138,0.206881,0.101727,0.072778,0.081564,0.114166,0.896459,-0.261779,...,1.928047,1.849282,0.004796,0.178719,0.357689,0.371424,0.208993,0.158744,0.529901,0.546505
2023-03-01 00:03:00,0.072944,-0.436590,-0.102483,0.017551,0.007149,-0.021681,-0.012936,0.019634,0.732634,-0.535845,...,1.928621,1.849608,0.004398,0.172967,0.351832,0.368358,0.208416,0.158524,0.529534,0.357703
2023-03-01 00:04:00,0.173820,-0.213489,0.096067,0.215709,0.107133,0.078976,0.087818,0.120426,0.763537,-0.430945,...,1.927084,1.848950,0.004008,0.167391,0.346066,0.365314,0.207839,0.158304,0.529167,0.362452


In [9]:
X_train = processed_train_df.drop(columns=["label"]).values
y_train = processed_train_df["label"].values

print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")

X_train: (525887, 890)
y_train: (525887,)


### Model Training

In [10]:
reg = xgb.XGBRegressor(n_estimators = 200,max_depth = 6,booster = "gbtree")
reg.fit(X_train,y_train)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)

In [19]:
train_df["dr_features"] = reg.predict(X_train)
train_df.head()

new_train_df = pd.DataFrame({"dr_features":train_df["dr_features"],
                             "label":train_df["label"],
                             "bid_qty":train_df["bid_qty"],
                             "ask_qty":train_df["ask_qty"],
                             "sell_qty":train_df["sell_qty"],
                             "volume":train_df["volume"],
                             "buy_qty":train_df["buy_qty"]})
new_train_df.head()

,dr_features,label,bid_qty,ask_qty,sell_qty,volume,buy_qty
timestamp,,,,,,,
2023-03-01 00:00:00,1.067519,0.562539,15.283,8.425,44.984,221.389,176.405
2023-03-01 00:01:00,1.131456,0.533686,38.590,2.336,321.950,847.796,525.846
2023-03-01 00:02:00,0.991776,0.546505,0.442,60.250,136.369,295.596,159.227
2023-03-01 00:03:00,0.836962,0.357703,4.865,21.016,124.963,460.705,335.742
2023-03-01 00:04:00,0.785671,0.362452,27.158,3.451,44.407,142.818,98.411


In [20]:
new_train_df.to_csv(os.path.join("Data","preprocessed_train_df.csv"))